<a href="https://colab.research.google.com/github/naveedkhalid091/Learn_Agentic_AI/blob/main/step02_generative_ai_for_beginners/02(e)Function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U -q "google-generativeai>=0.7.2"

In [3]:
from google.colab import userdata
import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

## **Set up a model with tools:**

This example uses 3 functions that control a simple hypothetical lighting system. Using these functions requires them to be called in a specific order. For example, you must turn the light system on before you can change color.

While you can pass these directly to the model and let it try to call them correctly, specifying the function_calling_config gives you precise control over the functions that are available to the model.




In [4]:
def enable_lights():
    """Turn on the lighting system."""  # telling to LLM regarding this function in tripple qutation.
    print("LIGHTBOT: Lights enabled.")


def set_light_color(rgb_hex: str):
    """Set the light color. Lights must be enabled for this to work."""   # telling to LLM regarding this function in tripple qutation.
    print(f"LIGHTBOT: Lights set to {rgb_hex}.")


def stop_lights():
    """Stop flashing lights."""   # telling to LLM regarding this function in tripple qutation.
    print("LIGHTBOT: Lights turned off.")

light_controls = [enable_lights, set_light_color, stop_lights]
instruction = "You are a helpful lighting system bot. You can turn lights on and off, and you can set the color. Do not perform any other tasks."

model = genai.GenerativeModel(
    "models/gemini-1.5-pro",
    tools=light_controls,  # Functions calling via tools
    system_instruction=instruction
)

# Now your model is ready for function calling

In [5]:
chat=model.start_chat()

## Create a helper function in tools configurations:

The helper function takes two parameters:
- `modes`: In simpler string form.
- `allowed functions`:  iterable of allowed function names.

and transforms above two parameters into the precise configuration format required by the Gemini API.

Creating the manual configurations each time can lead to error. The helper function minimizes the mistakes by abstracting the structure in a single, reuseable function.